# Profundidad monocular con DepthNet

La detección de profundidad es útil para tareas como mapeo, navegación y detección de obstáculos; sin embargo, históricamente requería una cámara estéreo o una cámara RGB-D. Ahora hay DNN que pueden inferir la profundidad relativa de una sola imagen monocular (también conocida como profundidad mono). Consulte el documento [FastDepth](https://arxiv.org/abs/1903.03273) del MIT para conocer uno de estos enfoques para lograr esto mediante el uso de redes totalmente convolucionales (FCN).

![depthnet-0](https://raw.githubusercontent.com/dusty-nv/jetson-inference/dev/docs/images/depthnet-0.jpg)

``depthNet`` acepta una imagen de un solo color como entrada y genera el mapa de profundidad. El mapa de profundidad está coloreado para su visualización, pero el campo de profundidad sin procesar también es accesible para acceder directamente a las profundidades. ``depthNet`` está disponible para su uso desde Python y C++ .

--------------

## Uso de los programas precompilados de la Jetson

Aquí se pueden encontrar los códigos de los programas precompilados:
 * [C++](https://github.com/dusty-nv/jetson-inference/blob/master/examples/depthnet/depthnet.cpp)
 * [Python](https://github.com/dusty-nv/jetson-inference/blob/master/python/examples/depthnet.py)

--------------

## Modelos de estimación de pose pre-entrenados disponibles

A continuación, se muestran las redes de profundidad previamente entrenadas disponibles para descargar y el argumento ``--network`` asociado que ``depthNet`` utilizará para cargar los modelos previamente entrenados:

|Model|CLI argument|NetworkType enum|
|-------|----------|----------|
|Monodepth-Mobilenet|``fcn-mobilenet``|``??``|
|Monodepth-Resnet18|``fcn-resnet18``|``??``|
|Monodepth-Resnet50|``fcn-resnet50``|``??``|

## Uso de los programas precompilados de la Jetson

Además de las rutas de entrada/salida, hay algunas opciones de línea de comandos adicionales:

 * flag ``--network`` (opcional) cambia el modelo de profundidad  que se está utilizando (el valor predeterminado es ``fcn-mobilenet``)
 * flag ``--visualize`` (opcional) acepta combinaciones separada por comas de ``input`` y ``depth`` (el valor predeterminado es ``input,depth``)
 * flag ``--depth-size`` (opcional) escala el tamaño del mapa de profundidad en relación con la entrada (el valor predeterminado es ``1.0``)
 * flag ``--filter-mode`` (opcional) establece el tipo de filtrado (``point`` o ``linear``) utilizado para el muestreo superior (el valor predeterminado es ``linear``)
 * flag ``--colormap`` (opcional) establece el mapeo de colores que se utilizará durante la visualización (el valor predeterminado es ``viridis_inverted``)

Usar el flag ``--help`` para obtener más información

Ir a la carpeta con los programas precompilados con el siguiente comando

```
$ cd jetson-inference/build/aarch64/bin
```

### Procesamiento de una imagen

A continuación, obtenemos la profundidad en una imagen de ejemplo con el programa precompilado ``depthNet``, tanto en C++ como en Python. Si está utilizando el contenedor Docker, es recomendable guardar la imagen de salida en el directorio images/test. Estas imágenes se podrán ver fácilmente desde su dispositivo host en el directorio jetson-inference/data/images/test.

```
# C++
$ ./depthnet images/room_0.jpg images/test/depth_room_0.jpg

# Python
$ ./depthnet images/room_'.jpg images/test/depth_room_0.jpg
```

### Procesamiento de varias imágenes

Si queremos detectar varias imágenes

```
# C++
$ ./depthnet "images/room_*.jpg" images/test/depth_room_%i.jpg

# Python
$ ./depthnet.py "images/room_*.jpg" images/test/depth_room_%i.jpg
```

 > **nota**: cuando se usen asteriscos, hay que escribirlos siempre entre comillas ("*.jpg"). De lo contrario, el sistema operativo expandirá automáticamente la secuencia y modificará el orden de los argumentos en la línea de comandos, lo que puede resultar en que una de las imágenes de entrada sea sobrescrita por la salida.

### Red

Si queremos cambiar el tipo de red flag `--network` (por defecto `fcn-mobilenet`)

```
# C++
$ ./depthnet --network=fcn-mobilenet images/room_0.jpg images/test/depth_room_0.jpg  # fcn-mobilenet network
$ ./depthnet --network=fcn-resnet18 images/room_0.jpg images/test/depth_room_0.jpg   # resnet18-hand network
$ ./depthnet --network=fcn-resnet50 images/room_0.jpg images/test/depth_room_0.jpg   # resnet50-hand network

# Python
$ ./depthnet.py --network=fcn-mobilenet images/room_0.jpg images/test/depth_room_0.jpg  # fcn-mobilenet network
$ ./depthnet.py --network=fcn-resnet18 images/room_0.jpg images/test/depth_room_0.jpg   # resnet18-hand network
$ ./depthnet.py --network=fcn-resnet50 images/room_0.jpg images/test/depth_room_0.jpg   # resnet50-hand network
```

### Visualización

Se puede cambiar el modo de visualización es mediante el flag `--visualize`, se puede elegir ``input`` y ``depth`` (el valor predeterminado es ``input,depth``)

```
# C++
$ ./depthnet --visualize=input images/room_0.jpg images/test/depth_room_0.jpg   # Visualize input
$ ./depthnet --visualize=depth images/room_0.jpg images/test/depth_room_0.jpg   # Visualize depth

# Python
$ ./depthnet.py --visualize=input images/room_0.jpg images/test/depth_room_0.jpg   # Visualize input
$ ./depthnet.py --visualize=depth images/room_0.jpg images/test/depth_room_0.jpg   # Visualize depth
```

### Escalado de la profundidad

Se puede cambiar el tamaño del mapa de profundidad en relación con la entrada mediante el flag `--depth-size` (el valor predeterminado es ``1.0``). Cuanto más pequeño el valor, más pequeño es el escalado

```
# C++
$ ./depthnet --depth-size=1.0 images/room_0.jpg images/test/depth_room_0.jpg
$ ./depthnet --depth-size=0.5 images/room_1.jpg images/test/depth_room_1.jpg
$ ./depthnet --depth-size=2.0 images/room_2.jpg images/test/depth_room_2.jpg

# Python
$ ./depthnet.py --depth-size=1.0 images/room_0.jpg images/test/depth_room_0.jpg
$ ./depthnet.py --depth-size=0.5 images/room_1.jpg images/test/depth_room_1.jpg
$ ./depthnet.py --depth-size=2.0 images/room_2.jpg images/test/depth_room_2.jpg
```

### Tipo de filtrado

Se puede cambiar el tipo de filtrado (``point`` o ``linear``) utilizado para el muestreo superior mediante el flag `--filter-mode` (el valor predeterminado es ``linear``). Cuanto más pequeño el valor, más pequeña es la linea

```
# C++
$ ./depthnet --filter-mode=linear images/room_0.jpg images/test/depth_room_0.jpg
$ ./depthnet --filter-mode=point images/room_1.jpg images/test/depth_room_1.jpg

# Python
$ ./depthnet.py --filter-mode=linear images/room_0.jpg images/test/depth_room_0.jpg
$ ./depthnet.py --filter-mode=point images/room_1.jpg images/test/depth_room_1.jpg
```

### Colormap

Se puede cambiar el mapeo de colores que se utilizará durante la visualización mediante el flag `--colormap` (el valor predeterminado es ``viridis_inverted``)

```
# C++
$ ./depthnet --colormap=viridis_inverted images/room_0.jpg images/test/depth_room_0.jpg

# Python
$ ./depthnet.py --colormap=viridis_inverted images/room_0.jpg images/test/depth_room_0.jpg
```

### Segmentación de vídeos

Si se quiere procesar un videdo solo hay que indicarlo en la entrada

Para ello ejecutamos el docker montando la carpeta del SDK de VisionWorks

```
$ docker/run.sh --volume /usr/share/visionworks/sources/data:/videos
```

Y ya lo podemos procesar

```
# C++
$ ./depthnet /videos/cars.mp4 images/test/cars_depth.mp4

# Python
$ ./depthnet.py /videos/cars.mp4 images/test/cars_depth.mp4
```

--------------

## Crear un programa de clasificación en Python

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-depth-python
$ cd my-depth-python
$ touch my-depth.py
$ chmod +x my-depth.py
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-depth-python:/my-depth-python   # mounted inside the container to /my-depth-python
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-depth-python
```

Editar el archivo .py con el siguiente comando

```
$ nano my-depth.py
```

Para crear un programa como el precompilado escribimos el siguiente código

```Python
import jetson.inference
import jetson.utils

import argparse
import sys

# Add /jetson-inference/python/examples path for import depthnet_utils
sys.path.append('/jetson-inference/python/examples')
from depthnet_utils import depthBuffers

# parse the command line
parser = argparse.ArgumentParser(description="Mono depth estimation on a video/image stream using depthNet DNN.", 
                                 formatter_class=argparse.RawTextHelpFormatter, epilog=jetson.inference.depthNet.Usage() +
                                 jetson.utils.videoSource.Usage() + jetson.utils.videoOutput.Usage() + jetson.utils.logUsage())

parser.add_argument("input_URI", type=str, default="", nargs='?', help="URI of the input stream")
parser.add_argument("output_URI", type=str, default="", nargs='?', help="URI of the output stream")
parser.add_argument("--network", type=str, default="fcn-mobilenet", help="pre-trained model to load, see below for options")
parser.add_argument("--visualize", type=str, default="input,depth", help="visualization options (can be 'input' 'depth' 'input,depth'")
parser.add_argument("--depth-size", type=float, default=1.0, help="scales the size of the depth map visualization, as a percentage of the input size (default is 1.0)")
parser.add_argument("--filter-mode", type=str, default="linear", choices=["point", "linear"], help="filtering mode used during visualization, options are:\n  'point' or 'linear' (default: 'linear')")
parser.add_argument("--colormap", type=str, default="viridis-inverted", help="colormap to use for visualization (default is 'viridis-inverted')",
                                  choices=["inferno", "inferno-inverted", "magma", "magma-inverted", "parula", "parula-inverted", 
                                           "plasma", "plasma-inverted", "turbo", "turbo-inverted", "viridis", "viridis-inverted"])

try:
	opt = parser.parse_known_args()[0]
except:
	print("")
	parser.print_help()
	sys.exit(0)

# load the segmentation network
net = jetson.inference.depthNet(opt.network, sys.argv)

# create buffer manager
buffers = depthBuffers(opt)

# create video sources & outputs
input = jetson.utils.videoSource(opt.input_URI, argv=sys.argv)
output = jetson.utils.videoOutput(opt.output_URI, argv=sys.argv)

# process frames until user exits
while True:
    # capture the next image
    img_input = input.Capture()

    # allocate buffers for this size image
    buffers.Alloc(img_input.shape, img_input.format)

    # process the mono depth and visualize
    net.Process(img_input, buffers.depth, opt.colormap, opt.filter_mode)

    # composite the images
    if buffers.use_input:
        jetson.utils.cudaOverlay(img_input, buffers.composite, 0, 0)
        
    if buffers.use_depth:
        jetson.utils.cudaOverlay(buffers.depth, buffers.composite, img_input.width if buffers.use_input else 0, 0)

    # render the output image
    output.Render(buffers.composite)

    # update the title bar
    output.SetStatus("{:s} | {:s} | Network {:.0f} FPS".format(opt.network, net.GetNetworkName(), net.GetNetworkFPS()))

    # print out performance info
    jetson.utils.cudaDeviceSynchronize()
    net.PrintProfilerTimes()

    # exit on input/output EOS
    if not input.IsStreaming() or not output.IsStreaming():
        break

```

Ejecutar el programa con el siguiente comando

```
$ python3 my-depth.py /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado

--------------

## Crear un programa ded clasificación en C++

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-depth-cpp
$ cd my-depth-cpp
$ touch my-depth.cpp
$ chmod +x my-depth.cpp
$ touch CMakeLists.txt
$ chmod +x CMakeLists.txt
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-depth-cpp:/my-depth-cpp   # mounted inside the container to /my-depth-cpp
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-depth-cpp
```

Editar el archivo.cpp con el siguiente comando

```
$ nano my-depth.cpp
```

Para crear un programa como el precompilado escribimos el siguiente código

```C++
#include <jetson-utils/videoSource.h>
#include <jetson-utils/videoOutput.h>

#include <jetson-utils/cudaFont.h>
#include <jetson-utils/cudaOverlay.h>
#include <jetson-utils/cudaMappedMemory.h>

#include <jetson-inference/depthNet.h>

#include <signal.h>


bool signal_recieved = false;

void sig_handler(int signo)
{
	if( signo == SIGINT )
	{
		printf("received SIGINT\n");
		signal_recieved = true;
	}
}

int usage()
{
	printf("usage: depthnet [--help] [--network NETWORK]\n");
	printf("                [--colormap COLORMAP] [--filter-mode MODE]\n");
	printf("                [--visualize VISUAL] [--depth-size SIZE]\n");
	printf("                input_URI [output_URI]\n\n");
	printf("Mono depth estimation on a video/image stream using depthNet DNN.\n\n");
	printf("See below for additional arguments that may not be shown above.\n\n");
	printf("optional arguments:\n");
	printf("  --help            show this help message and exit\n");
	printf("  --network=NETWORK pre-trained model to load (see below for options)\n");
	printf("  --visualize=VISUAL controls what is displayed (e.g. --visualize=input,depth)\n");
	printf("                     valid combinations are:  'input', 'depth' (comma-separated)\n");
	printf("  --depth-size=SIZE  scales the size of the depth map visualization, as a\n");
	printf("                     percentage of the input size (default is 1.0)\n");
	printf("  --filter-mode=MODE filtering mode used during visualization,\n");
	printf("                     options are:  'point' or 'linear' (default: 'linear')\n");
	printf("  --colormap=COLORMAP depth colormap (default is 'viridis-inverted')\n");
	printf("                      options are:  'inferno', 'inferno-inverted',\n");
	printf("                                    'magma', 'magma-inverted',\n");
	printf("                                    'parula', 'parula-inverted',\n");
	printf("                                    'plasma', 'plasma-inverted',\n");
	printf("                                    'turbo', 'turbo-inverted',\n");
	printf("                                    'viridis', 'viridis-inverted'\n\n");
	printf("positional arguments:\n");
	printf("    input_URI       resource URI of input stream  (see videoSource below)\n");
	printf("    output_URI      resource URI of output stream (see videoOutput below)\n\n");

	printf("%s", depthNet::Usage());
	printf("%s", videoSource::Usage());
	printf("%s", videoOutput::Usage());
	printf("%s", Log::Usage());
	
	return 0;
}


//
// depth map buffers
//
typedef uchar3 pixelType;        // this can be uchar3, uchar4, float3, float4

pixelType* imgDepth = NULL;      // colorized depth map image
pixelType* imgComposite = NULL;  // original image with depth map next to it

int2 inputSize;
int2 depthSize;
int2 compositeSize;

// allocate depth map & output buffers
bool allocBuffers( int width, int height, uint32_t flags, float depthScale )
{
	// check if the buffers were already allocated for this size
	if( imgDepth != NULL && width == inputSize.x && height == inputSize.y )
		return true;

	// free previous buffers if they exit
	CUDA_FREE_HOST(imgDepth);
	CUDA_FREE_HOST(imgComposite);

	// allocate depth map
	inputSize = make_int2(width, height);
	depthSize = make_int2(width * depthScale, height * depthScale);
	
	if( !cudaAllocMapped(&imgDepth, depthSize) )
	{
		LogError("depthnet:  failed to allocate CUDA memory for depth map (%ix%i)\n", depthSize.x, depthSize.y);
		return false;
	}

	// allocate composite image
	compositeSize = make_int2(0,0);
	
	if( flags & depthNet::VISUALIZE_DEPTH )
	{
		compositeSize.x += depthSize.x;
		compositeSize.y = depthSize.y;
	}
	
	if( flags & depthNet::VISUALIZE_INPUT )
	{
		compositeSize.x += inputSize.x;
		compositeSize.y = inputSize.y;
	}
	
	if( !cudaAllocMapped(&imgComposite, compositeSize) )
	{
		LogError("depthnet:  failed to allocate CUDA memory for composite image (%ix%i)\n", compositeSize.x, compositeSize.y);
		return false;
	}

	return true;
}


int main( int argc, char** argv )
{
	/*
	 * parse command line
	 */
	commandLine cmdLine(argc, argv);

	if( cmdLine.GetFlag("help") )
		return usage();

	
	/*
	 * attach signal handler
	 */
	if( signal(SIGINT, sig_handler) == SIG_ERR )
		LogError("can't catch SIGINT\n");


	/*
	 * create input stream
	 */
	videoSource* input = videoSource::Create(cmdLine, ARG_POSITION(0));

	if( !input )
	{
		LogError("depthnet:  failed to create input stream\n");
		return 0;
	}


	/*
	 * create output stream
	 */
	videoOutput* output = videoOutput::Create(cmdLine, ARG_POSITION(1));
	
	if( !output )
		LogError("depthnet:  failed to create output stream\n");
	

	/*
	 * create mono-depth network
	 */
	depthNet* net = depthNet::Create(cmdLine);

	if( !net )
	{
		LogError("depthnet:   failed to initialize depthNet\n");
		return 0;
	}

	// parse the desired colormap
	const cudaColormapType colormap = cudaColormapFromStr(cmdLine.GetString("colormap", "viridis-inverted"));

	// parse the desired filter mode
	const cudaFilterMode filterMode = cudaFilterModeFromStr(cmdLine.GetString("filter-mode"));

	// parse the visualization flags
	const uint32_t visualizationFlags = depthNet::VisualizationFlagsFromStr(cmdLine.GetString("visualize"));
	
	// get the depth map size scaling factor
	const float depthScale = cmdLine.GetFloat("depth-size", 1.0);


	/*
	 * processing loop
	 */
	while( !signal_recieved )
	{
		// capture next image image
		pixelType* imgInput = NULL;

		if( !input->Capture(&imgInput, 1000) )
		{
			// check for EOS
			if( !input->IsStreaming() )
				break;

			LogError("depthnet:  failed to capture next frame\n");
			continue;
		}

		// allocate buffers for this size frame
		if( !allocBuffers(input->GetWidth(), input->GetHeight(), visualizationFlags, depthScale) )
		{
			LogError("depthnet:  failed to allocate output buffers\n");
			continue;
		}
		
		// infer the depth and visualize the depth map
		if( !net->Process(imgInput, inputSize.x, inputSize.y, 
					   imgDepth, depthSize.x, depthSize.y, 
					   colormap, filterMode) )
		{
			LogError("depthnet-camera:  failed to process depth map\n");
			continue;
		}

		// overlay the images into composite output image
		if( visualizationFlags & depthNet::VISUALIZE_INPUT )
			CUDA(cudaOverlay(imgInput, inputSize, imgComposite, compositeSize, 0, 0));
		
		if( visualizationFlags & depthNet::VISUALIZE_DEPTH )
			CUDA(cudaOverlay(imgDepth, depthSize, imgComposite, compositeSize, (visualizationFlags & depthNet::VISUALIZE_INPUT) ? inputSize.x : 0, 0));
		
		// render outputs
		if( output != NULL )
		{
			output->Render(imgComposite, compositeSize.x, compositeSize.y);

			// update the status bar
			char str[256];
			sprintf(str, "TensorRT %i.%i.%i | %s | Network %.0f FPS", NV_TENSORRT_MAJOR, NV_TENSORRT_MINOR, NV_TENSORRT_PATCH, net->GetNetworkName(), net->GetNetworkFPS());
			output->SetStatus(str);

			// check if the user quit
			if( !output->IsStreaming() )
				signal_recieved = true;
		}

		// wait for the GPU to finish		
		CUDA(cudaDeviceSynchronize());

		// print out timing info
		net->PrintProfilerTimes();
	}
	

	/*
	 * destroy resources
	 */
	LogVerbose("depthnet:  shutting down...\n");
	
	SAFE_DELETE(input);
	SAFE_DELETE(output);
	SAFE_DELETE(net);
	
	CUDA_FREE_HOST(imgDepth);
	CUDA_FREE_HOST(imgComposite);

	LogVerbose("depthnet:  shutdown complete.\n");
	return 0;
}

```

Editar el CMakeList.txt con lo siguiente

```
# require CMake 2.8 or greater
cmake_minimum_required(VERSION 2.8)

# declare my-depth project
project(my-depth)

# import jetson-inference and jetson-utils packages.
# note that if you didn't do "sudo make install"
# while building jetson-inference, this will error.
find_package(jetson-utils)
find_package(jetson-inference)

# CUDA and Qt4 are required
find_package(CUDA)
find_package(Qt4)

# setup Qt4 for build
include(${QT_USE_FILE})
add_definitions(${QT_DEFINITIONS})

# compile the my-depth program
cuda_add_executable(my-depth my-depth.cpp)

# link my-depth to jetson-inference library
target_link_libraries(my-depth jetson-inference)
```

Compilar el código con los siguientes comandos

```
$ cmake .
$ make
```

Ejecutar el programa con el siguiente comando

```
$ ./my-depth /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado